In [5]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_Project


In [6]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import numpy as np
CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 43
MDs = create_MDs(config, seed=seed)

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 10, 'l': 0.8, 'cn': 0.019204365311182317, 'Fn': 1.7181330785589628, 'kn': 0.36669548209299413, 'omega_n': 1, 'Rn': 6.593466928446109}
{'s': 10, 'l': 0.8, 'cn': 0.07873099927588781, 'Fn': 1.8321804261960515, 'kn': 0.5705811061417019, 'omega_n': 1, 'Rn': 2.0511786712565887}
{'s': 10, 'l': 0.8, 'cn': 0.04159600147448047, 'Fn': 2.104094237257333, 'kn': 0.42721056292993964, 'omega_n': 1, 'Rn': 2.7066179693116306}
{'s': 10, 'l': 0.8, 'cn': 0.027682319670749703, 'Fn': 1.3099788960945193, 'kn': 0.45804823550910434, 'omega_n': 1, 'Rn': 3.202507720360912}
{'s': 10, 'l': 0.8, 'cn': 0.07791513181958891, 'Fn': 2.442930182870776, 'kn': 0.49268845719756904, 'omega_n': 1, 'Rn': 10.55394502033278}
{'lambda0': 50, 'D0': 4, 'theta': 5, 'o': 10, 'omega_0': 2}


In [7]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")

Optimal λ: [ 9.98967099  9.99688304 10.01407975  9.96385706 10.03550917]
Optimal p: [1.71812995 1.8321804  2.10409316 1.30997834 2.44292994]
Optimal r: [35.15252507 35.55724771 35.43470623 35.0601437  35.89979582]
Optimal Dmax: 2.051244742892487


In [8]:
lamb, p, Dmax = NBS.ADMM(esp, MDs)
# print(lamb)
# print(p)
# print(Dmax)
r = NBS.negotiation(esp, MDs,lamb, p, Dmax)
print(r)

lamb: [3.82535315e+262 2.75425377e+278 3.33728903e+019 8.79415321e+000
 8.74568578e-001] [4.09664289e+05 1.19571890e+04 4.36667479e+08 9.34584884e+03
 2.29778466e-09]
Dmax: 3.3537343615989497 [3119.86424381 3126.73619149 3126.73619149 3133.60813916 3532.18110423]
p: [5.17238563e-12 4.67852548e-07 3.92378225e-02 2.28049126e-02
 2.44293018e+00]
[33.10243639 33.10243659 33.11350656 33.11151751 35.12024022]
